In [21]:
import pandas as pd
import numpy as np

clean_car_features = pd.read_csv('../datasets/clean_car_features.csv')

In [22]:
# Feature Selection
#  "Make," "Model," "Year," "Engine Fuel Type," "Transmission Type," "Driven_Wheels," "Market Category," "Vehicle Size," and "Vehicle Style"

useful_features = ["Make", "Model", "Year", "Engine Fuel Type", "Transmission Type", "Driven_Wheels", "Market Category", "Vehicle Size", "Vehicle Style","Engine HP", "Engine Cylinders", "Number of Doors", "MSRP"]
numerical_features = ["Year", "Engine HP", "Engine Cylinders", "Number of Doors", "MSRP"]


In [23]:
# Text Data Processing
import string

# Function to clean and preprocess a text
def preprocess_text(text):
    # Step 1: Convert to lowercase
    text = text.lower()
    
    # Step 2: Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    
    # Step 3: Remove words containing numbers
    text = ' '.join([word for word in text.split() if not any(char.isdigit() for char in word)])
    
    # Step 4: Remove stop words (You would need a list of stop words for this step)
    # Example: text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text

processed_car_features = clean_car_features.copy()  # Copy the original DataFrame

# Replace the ',' with a space in the Market Category column
processed_car_features['Market Category'] = processed_car_features['Market Category'].str.replace(',', ' ')

# Apply the preprocessing function to each row of the DataFrame
processed_car_features['combined_text'] = processed_car_features[useful_features].apply(lambda x: ' '.join(x.astype(str)), axis=1)
processed_car_features['combined_text'] = processed_car_features['combined_text'].apply(preprocess_text)


In [24]:
processed_car_features['combined_text'].head().values

array(['bmw series m premium unleaded required manual rear wheel drive factory tuner luxury highperformance compact coupe',
       'bmw series premium unleaded required manual rear wheel drive luxury performance compact convertible',
       'bmw series premium unleaded required manual rear wheel drive luxury highperformance compact coupe',
       'bmw series premium unleaded required manual rear wheel drive luxury performance compact coupe',
       'bmw series premium unleaded required manual rear wheel drive luxury compact convertible'],
      dtype=object)